In [ ]:
# comstock60min -- zeroshot
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import time
from aeon.datasets import write_to_ts_file

# Load the data
df = pd.read_csv("/home/user/Downloads/Kajeeth/UniTS_Model/dataset/Comstock_1/60_min/comstock_60min_small.csv")
df = df.drop("Timestamp", axis=1)  # Drop Timestamp column
X = df.T  

# Define the appliances
appliance_name = ['cooling_ON', 'fans_ON', 'heat_rejection_ON', 'heating_ON', 'refrigeration_ON', 'water_systems_ON']

# Load labels
df_label_all = pd.read_csv("/home/user/Downloads/Kajeeth/UniTS_Model/dataset/Comstock_1/60_min/comstock_60min_labels.csv")
df_label = df_label_all[appliance_name]
y = df_label.values  

for appliance in appliance_name:
    start_time = time.time()  # Start time for each appliance
    y = df_label_all[appliance]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    X_train = np.array(X_train)[:, np.newaxis, :]
    X_test = np.array(X_test)[:, np.newaxis, :]
    y_train = np.array(y_train)
    y_test = np.array(y_test)
        
    # Write the training and testing time series files for the current appliance
    write_to_ts_file(X_train, '/home/user/Downloads/UniTS_Model/dataset/Comstock_1/60_min/temp/', y_train, problem_name=f'Comstock_TRAIN.ts', header=None, regression=False)
    write_to_ts_file(X_test, '/home/user/Downloads/UniTS_Model/dataset/Comstock_1/60_min/temp/', y_test, problem_name=f'Comstock_TEST.ts', header=None, regression=False)
    
    !bash ./scripts/few_shot_newdata/UniTS_finetune_few_shot_newdata_pct100.sh
    
    elapsed_time = time.time() - start_time
    
    print(f"Time series files for {appliance} written successfully!")
    print(f"Time taken for {appliance}: {elapsed_time:.2f} seconds.")